<a class="anchor" id="0"></a>
# [Tweet Sentiment Extraction](https://www.kaggle.com/c/tweet-sentiment-extraction)

## 1. Import libraries <a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns; sns.set(style='white')
from mpl_toolkits.mplot3d import Axes3D

from wordcloud import WordCloud
from sklearn.decomposition import PCA, TruncatedSVD
import math
import pickle

from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import *
import tokenizers
from sklearn.model_selection import StratifiedKFold
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import TimeDistributed
#from tensorflow.keras.layers import concatenate

import gc

pd.set_option('max_colwidth', 40)

## 2. Download data & FE <a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

Code from notebook https://www.kaggle.com/khoongweihao/tse2020-roberta-cnn-random-seed-distribution?scriptVersionId=34448972

In [2]:
MAX_LEN = 96
PATH = '../input/tf-roberta/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
EPOCHS = 3 # originally 3
BATCH_SIZE = 32 # originally 32
PAD_ID = 1
SEED = 88888
LABEL_SMOOTHING = 0.1
tf.random.set_seed(SEED)
np.random.seed(SEED)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv').fillna('')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in Sa...,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put...","Sons of ****,",negative


In [3]:
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask_t[k,:len(enc.ids)+3] = 1

In [4]:
Dropout_new = 0.15     # originally 0.1
n_split = 5            # originally 5
lr = 3e-5              # originally 3e-5

## 3.2. Model training <a class="anchor" id="3.2"></a>

[Back to Table of Contents](#0.1)

In [5]:
import pickle

def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)


def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model

def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss


def build_model_base():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    #shapeミスった。
    x1 = tf.keras.layers.Conv1D(64, 2,padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(Dropout_new)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(64, 2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model

In [6]:
def build_model_drop_bef64():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dropout(Dropout_new)(x1)
    x1 = tf.keras.layers.Conv1D(64, 2,padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(Dropout_new)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dropout(Dropout_new)(x2) 
    x2 = tf.keras.layers.Conv1D(64, 2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)
    
    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model    

In [7]:
def build_model_192():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(192, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dropout(Dropout_new)(x1)
    x1 = tf.keras.layers.Conv1D(64, 2,padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(Dropout_new)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(192, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dropout(Dropout_new)(x2) 
    x2 = tf.keras.layers.Conv1D(64, 2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)
    
    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model    

In [8]:
def build_model_lstm():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x1 = Bidirectional(CuDNNLSTM(784, return_sequences=True, name='lstm_layer',))(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x2 = Bidirectional(CuDNNLSTM(784, return_sequences=True, name='lstm_layer',))(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model    

In [9]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [10]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask[k,:len(enc.ids)+3] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+2] = 1
        end_tokens[k,toks[-1]+2] = 1

In [11]:
%%time
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
preds_start1 = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end1 = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=n_split,shuffle=True,random_state=SEED)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    #gc.collect()#このガベージコレクトには意味があるのか?
    
    model, padded_model = build_model_base()
   
    weight_fn = '../input/0716-lucky-model/%s-roberta-%i.h5'%(VER,fold)
    
    print('Loading model...')
    load_weights(model, weight_fn)

    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start1 += preds[0]/skf.n_splits
    preds_end1 += preds[1]/skf.n_splits
#以下を付け加える必要があるかを考察。    
K.clear_session()
gc.collect()

#########################
### FOLD 1
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 15s 4ms/sample
#########################
### FOLD 2
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 11s 3ms/sample
#########################
### FOLD 3
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 11s 3ms/sample
#########################
### FOLD 4
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 11s 3ms/sample
#########################
### FOLD 5
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 12s 3ms/sample
CPU times: user 1min 16s, sys: 11.3 s, total: 1min 27s
Wall time: 1min 41s


785

In [12]:
# %%time
# jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
# preds_start2 = np.zeros((input_ids_t.shape[0],MAX_LEN))
# preds_end2 = np.zeros((input_ids_t.shape[0],MAX_LEN))

# skf = StratifiedKFold(n_splits=n_split,shuffle=True,random_state=SEED)
# for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

#     print('#'*25)
#     print('### FOLD %i'%(fold+1))
#     print('#'*25)
    
#     K.clear_session()
#     #gc.collect()
    
#     model, padded_model = build_model_drop_bef64()
    
#     weight_fn = '../input/drop-bef64/add_drop_out_%s-roberta-%i.h5'%(VER,fold)
    
#     print('Loading model...')
#     load_weights(model, weight_fn)

#     print('Predicting Test...')
#     preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
#     preds_start2 += preds[0]/skf.n_splits
#     preds_end2 += preds[1]/skf.n_splits
    
# #以下を付け加える必要があるかを考察。    
# K.clear_session()
# gc.collect()

In [13]:
%%time
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
preds_start3 = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end3 = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=n_split,shuffle=True,random_state=SEED)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    #gc.collect()
    
    model, padded_model = build_model_192()
    
    weight_fn = '../input/768-192-64-conv-stable/%s-roberta-%i.h5'%(VER,fold)
    
    print('Loading model...')
    load_weights(model, weight_fn)

    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start3 += preds[0]/skf.n_splits
    preds_end3 += preds[1]/skf.n_splits

#以下を付け加える必要があるかを考察。    
K.clear_session()
gc.collect()

#########################
### FOLD 1
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 11s 3ms/sample
#########################
### FOLD 2
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 11s 3ms/sample
#########################
### FOLD 3
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 12s 3ms/sample
#########################
### FOLD 4
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 11s 3ms/sample
#########################
### FOLD 5
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 11s 3ms/sample
CPU times: user 1min 10s, sys: 8.44 s, total: 1min 19s
Wall time: 1min 27s


68462

In [14]:
%%time
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
preds_start5 = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end5 = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=n_split,shuffle=True,random_state=SEED)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    #gc.collect()
    
    model, padded_model = build_model_lstm()
    
    weight_fn = '../input/lstm768/%s-roberta-%i.h5'%(VER,fold)
    
    print('Loading model...')
    load_weights(model, weight_fn)

    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start5 += preds[0]/skf.n_splits
    preds_end5 += preds[1]/skf.n_splits

#以下を付け加える必要があるかを考察。    
K.clear_session()
gc.collect()

#########################
### FOLD 1
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 95s 27ms/sample
#########################
### FOLD 2
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 30s 8ms/sample
#########################
### FOLD 3
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 29s 8ms/sample
#########################
### FOLD 4
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 29s 8ms/sample
#########################
### FOLD 5
#########################
Loading model...
Predicting Test...
3534/3534 [==============================] - 29s 8ms/sample
CPU times: user 3min 43s, sys: 9.44 s, total: 3min 53s
Wall time: 4min 7s


72722

## 4. Submission <a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

Code from notebook https://www.kaggle.com/khoongweihao/tse2020-roberta-cnn-random-seed-distribution?scriptVersionId=34448972

In [15]:
preds_start = preds_start1*0.83 + preds_start3*0.1 + preds_start5*0.07
preds_end =  preds_end1*0.83 + preds_end3*0.1 + preds_end5*0.07

# preds_start = preds_start1
# preds_end = preds_end1

all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-2:b-1])
    all.append(st)

In [16]:
 test_raw = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')

In [17]:
print(test.shape, test_raw.shape)

(3534, 3) (3534, 3)


In [18]:
test['selected_text'] = all

In [19]:
import re

def rm_url(text): 
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    return ''.join(text)

def rm_underbar(text):
    text = re.sub('^_.{2,}?\s', '', text)
    return ''.join(text)

def post_process(s):
    a = re.findall('[^A-Za-z0-9]',s)
    b = re.sub('[^A-Za-z0-9]+', '', s)

    try:
        if a.count('.')==3:
            text = b + '. ' + b + '..'
        elif a.count('.')==4:
            text = b + '. ' + b + '..' + b + '...'
        elif a.count('!')==4:
            text = b + '! ' + b + '!! ' +  b + '!!!'
        else:
            text = s
        return text
    except:
        return text

In [20]:
# test_raw = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')
# for k in range(test.shape[0]):
#     if test.loc[k, 'sentiment'] == 'positive' or 'negative':
#         if len(test.loc[k, 'text'].split()) <= 3:
#             test.loc[k, 'selected_text'] = rm_url(test_raw.loc[k, 'text'])
# #     if test.loc[k, 'sentiment'] == 'neutral': 
# #         test.loc[k, 'selected_text'] = rm_url(test_raw.loc[k, 'text'])

# test['selected_text'] = test['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
# test['selected_text'] = test['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
# test['selected_text'] = test['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
# test['selected_text'] = test.apply(lambda x: post_process(x['selected_text']) if (len(str(x['selected_text']).split())==1) else x['selected_text'], axis=1)


In [21]:
test[['textID','selected_text']].to_csv('submission.csv',index=False)
test.sample(10)

,textID,text,sentiment,selected_text
3444,7d32d4866a,You are sooo lucky. My fiance is aw...,positive,lucky.
749,200f2b3566,you shouldnt have to reset more tha...,neutral,you shouldnt have to reset more tha...
3136,43c61eb7a3,I can`t find my camera,negative,i can`t find my camera
26,334954f215,"hey peoples, dont you just hate bein...",neutral,"hey peoples, dont you just hate bei..."
3208,2dfbe0b7fb,Hmmm. http://www.djhero.com/ is down,negative,down
2095,8986091fd3,wishes happy mothers` day to all mom...,positive,happy
843,7dd51bdec1,reading Evermore by Lynn Viehl.,neutral,reading evermore by lynn viehl.
172,5ff525b74b,Wondering if i cld make things any w...,negative,worse
100,f9382a1f7b,"oooh, sunshine! A patch of sunshine!...",neutral,"oooh, sunshine! a patch of sunshine..."
930,7720aca199,"No, that`s not right - I remember n...",neutral,"no, that`s not right - i remember n..."


There are a number of clear clusters that allow us to hope that we can improve the solution.

[Go to Top](#0)